# Client Churn Prediction
### CRISP-DM Cycle 4
---
The Top Bank company operates in Europe with a bank account as the main product, this product can keep client's salary and make payments. This account doesn't have any cost in the first 12 months, however, after that time trial, the client needs to rehire the bank for upcoming 12 months and redo this process every year. Recently the Analytics Team noticed that the churn rate is increasing.

As a Data Science Consultant, you need to create an action plan to decrease the number of churn customers and show the financial return on your solution.
At the end of your consultancy, you need to deliver to the TopBottom CEO a model in production, which will receive a customer base via API and return that same base with an extra column with the probability of each customer going into churn.
In addition, you will need to provide a report reporting your model's performance and the financial impact of your solution. Questions that the CEO and the Analytics team would like to see in their report:

1.  What is Top Bank's current Churn rate?
2.  How does the churn rate vary monthly?
3.  What is the performance of the model in classifying customers as churns
4.  What is the expected return, in terms of revenue, if the company uses its model to avoid churn from customers?

> Disclaimer: This is a fictional business case

## 0. Preparation

### 0.1 Imports & Settings

In [56]:
from IPython.core.display import HTML
from pathlib import Path
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import pickle
import duckdb
from src.app.utils.utils_business_translation import (
    churn_rate_per_month,
    top_clients,
    knapsack_solver,
)
from src.utils_ml import metrics_cv#, threshold_tuning_plot_single

In [57]:
def jupyter_settings():
    """
    Plots pre settings.
    """

    %matplotlib inline
    plt.style.use("seaborn-v0_8-whitegrid")
    plt.rcParams["figure.figsize"] = [25, 12]
    plt.rcParams["font.size"] = 24
    display(HTML("<style>.container {width:100% !important;}</style>"))
    sns.set()


jupyter_settings()

seed = 42

### 0.3 Path

In [58]:
# found the main project folders
path = Path().resolve().parent
data_path = path / "data"
model_path = path / "src/models"

### 0.4 Data

This dataset is available [here](https://www.kaggle.com/mervetorkan/churndataset).


**Data fields**

- **RowNumber**: the number of the columns
- **CustomerID**: unique identifier of clients
- **Surname**: client's last name
- **CreditScore**: clients credit score for the financial market
- **Geography**: the country of the client
- **Gender**: the gender of the client
- **Age**: the client's age
- **Tenure**: number of years the client is in the bank 
- **Balance**: the amount that the client has in their account 
- **NumOfProducts**: the number of products that the client bought 
- **HasCrCard**: if the client has a credit card 
- **IsActiveMember**: if the client is active (within the last 12 months) 
- **EstimateSalary**: estimate of annual salary of clients 
- **Exited**: if the client is a churn (*target variable*)

In [59]:
# Load data in duckdb
conn_path = str(data_path / "interim/churn.db")
conn = duckdb.connect(database=conn_path, read_only=False)
query = conn.execute("SELECT * FROM churn")
df_raw = pd.DataFrame(query.fetchdf())
conn.close()

# Load parquet and pkl
X_test_old = pd.read_parquet(data_path / "processed/X_test.parquet")
estimated_salary = X_test_old["estimated_salary"]
X_train = pd.read_parquet(data_path / "processed/X_train_hft.parquet")
X_test = pd.read_parquet(data_path / "processed/X_test_fs.parquet")
y_train = pd.read_pickle(data_path / "processed/y_train_hft.pkl")
y_test = pd.read_pickle(data_path / "processed/y_test.pkl")

## 7. Business Translation

In [60]:
model = pickle.load(open(str(model_path) + "/model.pkl", "rb"))
threshold = pickle.load(open(str(model_path) + "/threshold.pkl", "rb"))
yhat_proba = model.predict_proba(X_test)[:, 1]
yhat = (yhat_proba >= threshold).astype(int)

### 6.1 What is Top Bank's current Churn rate?

In [61]:
churn_rate = df_raw[df_raw["exited"] == 1]["exited"].count() / (len(df_raw))
round(churn_rate * 100, 2)

20.37

The current churn rate is 20.37%

### 6.2 How does the churn rate vary monthly?

In [62]:
monthly_churn_rate = churn_rate_per_month(df_raw)

print(f" The churn rate varies {round(monthly_churn_rate, 2)} per month.")

 The churn rate varies 8.33 per month.


### 6.3 What is the performance of the model in classifying customers as churns?


In [63]:
metrics_cv([model], X_train, y_train, threshold=0.47, verbose=False)

Please wait a moment - Doing CV
Finished, check the results


,Model_Name,Threshold,Balanced_Accuracy Mean,Balanced_Accuracy STD,Precision Mean,Precision STD,Recall Mean,Recall STD,F1 Score Mean,F1 Score STD,ROCAUC Mean,ROCAUC STD
0,CatBoostClassifier,0.47,0.779,0.009,0.478,0.017,0.776,0.011,0.591,0.014,0.779,0.779


In [64]:
threshold_tuning_plot_single(model, X_test, y_test, threshold);


### 6.4 What is the expected return, in terms of revenue, if the company uses its model to avoid churn from customers?

#### 6.4.1 Creating DataFrame

In [65]:
df_raw["estimated_salary_regular"] = estimated_salary

In [66]:
# df_raw is a dataframe before the transformations
salary_mean = round(df_raw["estimated_salary"].mean(), 2)

# Predictions and Results
y_test_frame = y_test.to_frame().reset_index(drop=True)
y_proba = (
    pd.DataFrame(yhat_proba).rename(columns={0: "probability"}).reset_index(drop=True)
)
y_predict = pd.DataFrame(yhat).rename(columns={0: "prediction"}).reset_index(drop=True)

# Estimated salary without mms
estimated_salary_frame = estimated_salary.to_frame().reset_index(drop=True)

# Creating a dataframe with the results
df_simulation = pd.concat(
    (y_test_frame, y_proba, y_predict, estimated_salary_frame), axis=1
)

# Verify threshold
df_simulation["threshold"] = df_simulation["probability"].apply(
    lambda x: "negative" if x <= 0.4 else "positive"
)

# Reorder columns
df_simulation = df_simulation[
    ["estimated_salary", "exited", "prediction", "probability", "threshold"]
]

In [67]:
salary_mean

100090.24

In [68]:
df_simulation.head()

,estimated_salary,exited,prediction,probability,threshold
0,94283.09,0,0,0.125404,negative
1,55072.93,0,0,0.350899,negative
2,105433.53,0,0,0.143586,negative
3,33950.08,0,0,0.259019,negative
4,168544.85,0,0,0.266519,negative


#### 6.4.2 Return per Client

Considering a return per client:
- *15%* if the `estimated_salary` is lower than avg;
- *20%* if the `estimated_salary` is equal to the avg and is also less than two times the avg;
- *25%* if the `estimated_salary` is two times higher or more than avg;

In [69]:
df_simulation["financial_return"] = df_simulation["estimated_salary"].apply(
    lambda x: x * 0.15
    if x < salary_mean
    else x * 0.2
    if salary_mean <= x < 2 * salary_mean
    else x * 0.25
)

return_clients = round(df_simulation["financial_return"].sum(), 2)
print(f"The return of all clients in this dataframe are: ${return_clients}")

The return of all clients in this dataframe are: $38210856.42


In [70]:
churn_loss = round(
    df_simulation[df_simulation["exited"] == 1]["financial_return"].sum(), 2
)
print(
    f"The bank is losing ${churn_loss}, that value represents {round ((churn_loss/return_clients)*100, 2)}% of the total return."
)

The bank is losing $7517032.21, that value represents 19.67% of the total return.


**Simulation 1 - Discount Ticket of $100**

With a budget of $10000.00, the bank can deliver $100 discount tickets to the top 100 clients with the highest probability of churn.

In [71]:
simulation_1 = top_clients(
    "Simulation 1",
    df_simulation,
    "probability",
    "prediction",
    "financial_return",
    churn_loss,
    100,
    100,
)

**Simulation 2 - Discount Ticket of $50**

With a budget of $10000.00, the bank can deliver $50 discount tickets to the top 200 clients with the highest probability of churn.

In [72]:
simulation_2 = top_clients(
    "Simulation 2",
    df_simulation,
    "probability",
    "prediction",
    "financial_return",
    churn_loss,
    200,
    50,
)

**Simulation 3 - knapsack with an incentive of $100**

Select the optimal combination of clients that maximize the total returned value , without exceeding the total constraint.

Using the 0-1 knapsack-problem with probabilities with a budget of $10000.00 and $100 tickets.

In [73]:
df_simulation_3 = df_simulation[df_simulation["prediction"] == 1].copy()
df_simulation_3["incentive"] = 100

simulation_3 = knapsack_solver(
    "Simulation 3",
    df_simulation_3,
    "prediction",
    "financial_return",
    churn_loss,
    10000,
    "incentive",
)

**Simulation 4 - knapsack with an incentive list**

Select the optimal combination of clients that maximize the total returned value , without exceeding the total constraint.

Using the 0-1 knapsack-problem with probabilities with a budget of $10000.00:
- p(churn) >= 0.99: Client that will leave
- 0.95 <= p(churn) < 0.99: Client with a high probability to stay with a $200 ticket;
- 0.90 <= p(churn) < 0.99: Client that might stay with a $100 ;
- p(churn) < 0.90: Client that might stay with a $50 ticket.



In [74]:
df_simulation_4 = df_simulation[df_simulation["prediction"] == 1].copy()

# list of available incentives as stated above
# the incentive of 20000 is for clients with p(churn) > 0.99, it will be discarded in the knapsack function
incentives_list = [200, 100, 50]
incentives = []
n = len(df_simulation_4)

# set incentive value according churn predicted probability
for i in range(n):
    entry = df_simulation_4.iloc[i]
    if entry["probability"] >= 0.95 and entry["probability"]:
        incentives.append(incentives_list[0])
    elif 0.90 <= entry["probability"] < 0.95:
        incentives.append(incentives_list[1])
    else:
        incentives.append(incentives_list[2])

df_simulation_4["incentive"] = incentives

simulation_4 = knapsack_solver(
    "Simulation 4",
    df_simulation_4,
    "prediction",
    "financial_return",
    churn_loss,
    10000,
    "incentive",
)

In [75]:
compare_df = pd.concat(
    [simulation_1, simulation_2, simulation_3, simulation_4], axis=0, ignore_index=True
)
compare_df

,Scenario,Recovered Revenue,Loss Recovered,Investment,Profit,ROI,Clients Recovered,Churn Reduction
0,Simulation 1,$1650351.21,21.95%,$10000,$1640351.21,16403.51%,93 clients,22.85%
1,Simulation 2,$3152913.76,41.94%,$10000,$3142913.76,31429.14%,163 clients,40.05%
2,Simulation 3,$1649498.2,21.94%,$10000,$1639498.2,16394.98%,44 clients,100.0%
3,Simulation 4,$2717398.24,36.15%,$10000,$2707398.24,27073.98%,80 clients,100.0%


In [76]:
compare_df.T

,0,1,2,3
Scenario,Simulation 1,Simulation 2,Simulation 3,Simulation 4
Recovered Revenue,$1650351.21,$3152913.76,$1649498.2,$2717398.24
Loss Recovered,21.95%,41.94%,21.94%,36.15%
Investment,$10000,$10000,$10000,$10000
Profit,$1640351.21,$3142913.76,$1639498.2,$2707398.24
ROI,16403.51%,31429.14%,16394.98%,27073.98%
Clients Recovered,93 clients,163 clients,44 clients,80 clients
Churn Reduction,22.85%,40.05%,100.0%,100.0%


### 6.5 Business Performance Summary
- The Top Bank's **churn rate is 20.37%**
- The monthly churn rate varies, on average, 8.33%
- The model has an accuracy of 77.9% and a recall of 81.2% in classifying clients as churn
- The bank is **losing *$7,517,032.21* in this dataframe because of the churn effect**
- The return of all clients in this dataframe are: *$38,210,856.42*
- Using the knapsack approach with an incentive list with tickets of $200, $100 and $50 depending on the probability to client's churn can give:
  - Recovered Revenue: *$2717398.24*
  - Churn Loss Recovered: **100%**
  - Investment: *$10000*
  - Profit: *$2,707,398.24*
  - ROI: **27,073.98%**
  - Potential clients recovered with the model: 87 clients 